In [1]:
#Definindo finais de semana
mes = "06"
lista_fds = ['01','02','08','09','15','16','22','23','29','30']

In [2]:
# Manipulação e Tratamento de dados
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#definindo quantidade de linhas e colunas visiveis
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Manipulação de Strings
import unidecode # => unidecode(string)
import re # Regex

# Repositorio - Diretorio
import os
import shutil

In [3]:
#######################-FUNÇÕES
def count_freq(data_frame, coluna):
    freq = df[coluna].value_counts()
    perc = df[coluna].value_counts(normalize = True)*100
    tabela = pd.DataFrame({"Freq":freq, "Perc":perc})
    return display(tabela)
        
def tabela_cont(linha, coluna):
    a = df.groupby([linha, coluna]).size().unstack()
    for i in a.columns:
        a[i].fillna(0, inplace = True)
        a[i] = a[i].apply(lambda x: int(x))
    return a
    print('='*100)

def tabela_cont_percent(linha, coluna, opcao):
    result = tabela_cont(linha, coluna)
    if opcao == 'Linha':
        return round(result.div(result.sum(axis=1), axis=0) * 100, 2)
    elif opcao == 'Coluna':
        return round(result.div(result.sum(), axis=1) * 100, 2)
    else:
        return 'Opção invalida'

In [4]:
def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        len_antigo = len(dataframe)
        
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe = dataframe.drop_duplicates(subset=['chat'])
        
        print(len(df1), " += ", len_antigo, "=>", len(dataframe))
        
        dataframe.to_excel(nome_arquivo, index=False)
    
    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print(len(dataframe)) 

In [5]:
def read_file(file_path):
    # 1. Ler o arquivo txt
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    
    # 2. Remover todas as linhas que contêm "Mensagem apagada"
    lines = [line for line in lines if "Mensagem apagada" not in line]
    
    # 3. Corrigir as mensagens que estão em várias linhas para que fiquem em uma única linha
    corrected_lines = []
    current_message = ""
    
    for line in lines:
        # Se a linha começa com o padrão de data e hora, ela é uma nova mensagem
        if re.match(r"^\[\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}\]", line):
            if current_message:  # Se há uma mensagem anterior, adicione-a à lista
                corrected_lines.append(current_message)
            current_message = line.strip()  # Comece uma nova mensagem
        else:
            current_message += " " + line.strip()  # Adicione o conteúdo da linha atual à mensagem em progresso
    
    # Adiciona a última mensagem, se houver
    if current_message:
        corrected_lines.append(current_message)
    
    # Salvando arquivo txt
    with open('processed.txt', "w", encoding="utf-8") as file:
        for line in corrected_lines:
            file.write(line + "\n")
    
    # Lendo arquivo txt e transformando em dataframe cada mensagem
    with open('processed.txt', 'r', encoding='utf-8') as file:
        content = file.read().splitlines()
    
    return content

In [6]:
df = pd.DataFrame({'chat': read_file('_chat.txt') })
#df['chat'] = df['chat'].apply(lambda x: x.replace('/05/2024, ','/05/2024 '))
df['controle'] = np.nan
df = df[['controle','chat']]
df_prob = pd.DataFrame(columns = ['controle','chat'])

print(len(df))
display(df.head())
display(df_prob)

679


,controle,chat
0,NaN,[19/06/2024 15:37:24] Sergio Zamith: 19/06 Ita...
1,NaN,[19/06/2024 15:47:24] Rodrigo Miyashiro Nunes ...
2,NaN,[19/06/2024 15:52:56] Pedro Paulo Silva Farah:...
3,NaN,[19/06/2024 15:54:34] Patrick Alexander Sauer ...
4,NaN,[19/06/2024 16:14:37] Antonio Alvaro Dias Lima...


,controle,chat


In [7]:
def sample_prob(nivel):
    '''
    data = df_prob[df_prob['controle'] == nivel]
    tamanho = len(data)
    try:
        if tamanho == 0:
            data_exibir = str('0 itens do prob' + nivel)
        if tamanho == 1:
            data_exibir = data['mensagem'].sample(1)
        if tamanho == 2:
            data_exibir = data['mensagem'].sample(2)
        if tamanho > 5:
            data_exibir = data['mensagem'].sample(5)
    except:
        data_exibir = 'erro'
    '''
    return 'nada'

# DIA QUE FOI ENVIADA A MENSAGEM

In [8]:
dia = []
for i in range(0, len(df['chat'])):
    dia.append(df['chat'][i][1:11]) #pegando exatamente aonde parte da data esta
df['dia'] = dia
#print(sorted(df['dia'].unique()))
print("Quantidade de mensagens por dia:")
count_freq(df, 'dia')

Quantidade de mensagens por dia:


,Freq,Perc
dia,,
20/06/2024,136,20.029455
24/06/2024,134,19.734904
21/06/2024,126,18.556701
22/06/2024,113,16.642121
25/06/2024,81,11.929308
19/06/2024,60,8.836524
23/06/2024,29,4.270987


# NOME DO ANESTESISTA

In [9]:
df['nome'] = np.nan
count = 0
for i in range(0, len(df['chat'])):
    loc = df['chat'][i].find(']')+1 #digito data
    loc2 = df['chat'][i][loc+1:].find(':')+1 #digito nome
    nome = df['chat'][i][loc+1:loc+loc2]
    nome = nome.upper().strip().replace("  ", " ")
    nome = unidecode.unidecode(nome)
    df['nome'][i] = nome
    count += 1
    #print(count, "->", unidecode.unidecode(chat[i][loc+1:loc+loc2].upper()))

print("Profissionais que mandaram mensagem:")
sorted(df['nome'].unique())

Profissionais que mandaram mensagem:


['ALESSANDRE LUIZ BRAGA JUNIOR',
 'ALEXANDRE AMARAL LUCAS',
 'ALEXANDRE BERNARDINO',
 'ALEXSANDER TAKASHI HAMADA',
 'ALINE FRANCHINI GOULART LIMA',
 'AMANDA KIYOKO SOUZA KOSSAKA',
 'ANA CLARA MOURAO BARRETO',
 'ANA JUNG DOCKO',
 'ANDRE AUGUSTO BELLINI',
 'ANDRE DA COSTA AGUIAR VENTURA',
 'ANDRE LUIS XAVIER FRANCO',
 'ANDREIA LOUREIRO MORI',
 'ANNA BEATRIZ ARANHA TREVISAN',
 'ANTONIO ALVARO DIAS LIMA DA CUNHA NETO',
 'ARTHUR OLIVEIRA FACCHINI',
 'ARTHUR RODRIGUES TORRELIO',
 'AUGUSTO DE CASTRO SANTOS QUEIROZ',
 'BIANCA PRISCILA BUNIOTTI',
 'BRIAN SOSSAI PACHECO',
 'BRUCE YE MAN CHOW',
 'BRUNA DAIBERT DANESIN',
 'BRUNO FERRAZ DE CAMPOS BARONE',
 'BRUNO STORNIOLI TAGUCHI LEMOS',
 'CAIO TULIO DUARTE',
 'CAMILA CHIARANTANO HARRISON',
 'CAMILA RESENDE DE PAULA',
 'CARLOS AUGUSTO TREVISAN',
 'CARLOS EDUARDO CORREA GRILO',
 'CARLOS EDUARDO GONDIM OLIVEIRA',
 'CAROLINE FIRMINO LEMES DA SILVA',
 'CLAUDIO MUELLER KAKUDA',
 'DANIEL MASUZAKI WONG',
 'DANIEL RODRIGUES DE OLIVEIRA',
 'DANIELLEN LINS 

# MENSAGENS ESCREVIDAS

In [10]:
mensagem = []
for i in range(0, len(df)):
    digito_data = df['chat'][i].find(']') #aonde acaba a data
    digito_nome = df['chat'][i][digito_data+1:].find(':')
    mensagem_escrevida = df['chat'][i][digito_data+digito_nome+3:]
    mensagem.append(mensagem_escrevida)
    
df['mensagem'] = mensagem
df['mensagem'] = df['mensagem'].apply(lambda x: x.upper() )
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('-',' - '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('(',' ( '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace(')',' ) '))

print('Mensagens Enviadas:')
df['mensagem'].sample(3)

Mensagens Enviadas:


451    23/06 ANÁLIA  -  AMANDA SAÍDA 12:00  ( D3 VILL...
599                 24/06 ITAIM RAFAEL TADEU SAÍDA 21:10
177                     20/06 OSASCO KAWAURA SAÍDA 21:20
Name: mensagem, dtype: object

# DATA DO PLANTÃO

In [11]:
df['data'] = np.nan

for i in range(0, len(df)):
    lista_datas = re.findall(r'[0-9]{1,2}\/[0-9]{1,2}', df['mensagem'][i]) #regex dd/mm
    quantidade_itens = len(lista_datas)
    
    if quantidade_itens == 1: #capturar apenas registros com uma data apenas
        df['data'][i] = lista_datas[0]
    else: #restante com nenhuma ou mais de uma data
        df['controle'][i] = 'DATA_DO_PLANTÃO'

#salvando
df_prob = df[df['controle'] == 'DATA_DO_PLANTÃO']
print("data probs:", len(df_prob))
sample_prob('DATA_DO_PLANTÃO')

df = df[df['controle'] != 'DATA_DO_PLANTÃO']
print("datas ok:", len(df)) 

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

count_freq(df, 'data')
#print("Dias mensionados nas mensagens:")
#print(sorted(df['data'].unique()))

data probs: 0
datas ok: 679


,Freq,Perc
data,,
24/06,135,19.882180
20/06,129,18.998527
21/06,122,17.967599
22/06,110,16.200295
25/06,74,10.898380
19/06,69,10.162003
23/06,26,3.829161
20/6,2,0.294551
21/6,2,0.294551


# CAPTURANDO UNIDADE

In [12]:
df['unidade'] = ''
df['Q_unidades'] = ''

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    
    #Unidades escrita com a bunda
    lista_avi = ["CENTRAL","AVICENA","AVICENNA","AVICCENNA"]
    for i_avi in lista_avi:
        mensagem = mensagem.replace(i_avi, 'AVICCENA')
        
    mensagem = mensagem.replace('Sta Isabel', 'SANTA')
    
    lista_unidades = re.findall(r'ALPHA|ANALIA|AVICCENA|ITAIM|JABA|MATERNIDADE|MORUMBI|OSASCO|VILLA|VILA|VNS|SANTA', mensagem)
    quantidade_unidades = len(lista_unidades)
    
    df['Q_unidades'][i] = quantidade_unidades
    
    if quantidade_unidades == 1: #capturando apenas com uma unidade
        df['unidade'][i] = lista_unidades[0].replace('  ',' ').upper().strip()
    elif quantidade_unidades > 1: #com mais de uma unidade
        df['unidade'][i] = lista_unidades
        df['controle'][i] = 'DESLOCADO'
    else: #sem nenhuma unidade
        df['controle'][i] = 'UNIDADES'
        
print("quantidades de unidades presentes nas mensagens:")
count_freq(df, 'Q_unidades')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'UNIDADES']])
print("unidades probs:", len(df_prob))
sample_prob('UNIDADES')

df = df[df['controle'] != 'UNIDADES']

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
#print("unidade ok:", len(df)) 
#print("unidade ok deslocado:", len(df[df['controle'] == 'DESLOCADO']))

quantidades de unidades presentes nas mensagens:


,Freq,Perc
Q_unidades,,
1,618,91.016200
2,54,7.952872
0,6,0.883652
3,1,0.147275


unidades probs: 6


# RETIRANDO REGISTROS DE DESLOCADOS, DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE

In [13]:
#preparando para reconhecer corretamente
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCADA','DESLOCADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCAMENTO','DESLOCADO'))

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DESLOCADO', mensagem)
    quantidade_observacoes = len(observacoes)
    
    if quantidade_observacoes > 0:
        df['controle'][i] = 'DESLOCADO'
        
count_freq(df, 'controle')
# OBS: Plantões de deslocado não são mais excluido

,Freq,Perc
controle,,
DESLOCADO,61,100.0


In [14]:
#RETIRANDO REGISTROS DE DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE OU DISPENSA
df['controle2'] = np.nan
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSADA','DISPENSADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSA ','DISPENSADO'))

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DISPENSADO|SAUDE|DISTANCIA|D[0-9]', mensagem)
    quantidade_observacoes = len(observacoes)

    if quantidade_observacoes >= 1:
        df['controle'][i] = 'DISP_SAUDE_DIST'
        if quantidade_observacoes == 1:
            df['controle2'][i] = observacoes[0]
        
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'DISP_SAUDE_DIST']])
print("unidades probs:", len(df_prob))
sample_prob('DISP_SAUDE_DIST')

df = df[df['controle'] != 'DISP_SAUDE_DIST']
print("unidade ok:", len(df)) 

print(df_prob['mensagem'][df_prob['controle'] == 'DISP_SAUDE_DIST'].sample(2))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,54,65.853659
DISP_SAUDE_DIST,28,34.146341


unidades probs: 34
unidade ok: 645
487                          24/06 ANALIA ANA DISPENSADO
447    23/06 JABAQUARA  -  M TERESA SAÍDA  12:55  ( D...
Name: mensagem, dtype: object


# VERIFICANDO SE É EXTRA

In [15]:
df["extra"] = np.nan

for i in range(0, len(df)):
    if 'EXTRA' in df['mensagem'][i].upper(): #se conter a palavra "extra"
        df['extra'][i] = 'extra' #marque na coluna extra como extra
        
print("Quantidade de registros Extra - Voluntario:")
count_freq(df, 'extra')      

Quantidade de registros Extra - Voluntario:


,Freq,Perc
extra,,
extra,5,100.0


# Tipo entrada e saida:

In [16]:
#Indentificando se é um registro de entrada ou saida
df['tipo'] = np.nan

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    entradas_saidas = re.findall(r'ENTRADA|SAIDA', mensagem)
    quantidade_entradas_saidas = len(entradas_saidas)
    
    if quantidade_entradas_saidas != 1: #capturando apenas com uma informação escrita
        df['controle'][i] = 'SEM_ENTRADA_OU_SAIDA'
    else: #restante com mais de uma informação
        df['tipo'][i] = entradas_saidas[0]
        
count_freq(df, 'tipo')
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'SEM_ENTRADA_OU_SAIDA']])
print("unidades probs:", len(df_prob))
sample_prob('SEM_ENTRADA_OU_SAIDA')

df = df[df['controle'] != 'SEM_ENTRADA_OU_SAIDA']
print("unidade ok:", len(df)) 

#print(df_prob['mensagem'][df_prob['controle'] == 'SEM_ENTRADA_OU_SAIDA'].sample(1))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
print("ok:", len(df)) 

,Freq,Perc
tipo,,
SAIDA,394,61.658842
ENTRADA,245,38.341158


,Freq,Perc
controle,,
DESLOCADO,54,90.0
SEM_ENTRADA_OU_SAIDA,6,10.0


unidades probs: 40
unidade ok: 639
ok: 639


# Data:

In [17]:
#retirando a data da mensagem para não atrapalhar no reconhecimento de hora
df['hora'] = np.nan

for i in range(0, len(df)):
    df['hora'][i] = df['mensagem'][i].replace(df['data'][i], '').strip() #substituindo a data por vazio

In [18]:
#arrumando data d/mm (6/10, não 06/10)
for i in range(0, len(df)):
    if df['data'][i][1] == '/':
        df['data'][i] = '0'+df['data'][i]
df['data'].unique()

array(['19/06', '17/06', '20/06', '06/06', '20/6', '21/06', '21/6',
       '21/05', '22/06', '22/03', '22/6', '23/06', '24/06', '24/6',
       '24/05', '25/06', '25/05', '25/6'], dtype=object)

In [19]:
#somando informações restantes na data: dd/MM/yyyy
for i in range(0, len(df)):
    df['data'][i] = df['data'][i][0:2] + "/"+ mes +"/2024" 
    
count_freq(df, 'data')

,Freq,Perc
data,,
24/06/2024,133,20.813772
20/06/2024,128,20.031299
21/06/2024,117,18.309859
22/06/2024,108,16.901408
25/06/2024,73,11.424100
19/06/2024,66,10.328638
23/06/2024,12,1.877934
17/06/2024,1,0.156495
06/06/2024,1,0.156495


In [20]:
df['sem_ou_fds'] = 'semana'

for i in lista_fds:
    datinha = i+"/"+mes+"/2024"
    df['sem_ou_fds'][df['data'] == datinha] = 'fds'

count_freq(df, 'sem_ou_fds')

,Freq,Perc
sem_ou_fds,,
semana,519,81.220657
fds,120,18.779343


In [21]:
print(df.groupby(['data', 'sem_ou_fds']).size().unstack())

sem_ou_fds    fds  semana
data                     
06/06/2024    NaN     1.0
17/06/2024    NaN     1.0
19/06/2024    NaN    66.0
20/06/2024    NaN   128.0
21/06/2024    NaN   117.0
22/06/2024  108.0     NaN
23/06/2024   12.0     NaN
24/06/2024    NaN   133.0
25/06/2024    NaN    73.0


In [22]:
#df['data'].iloc[180] = '26/09/2022'

# Hora:

In [23]:
#substituindo "h" por ":" para capturar horarios 11h22
df['hora'] = df['hora'].apply(lambda x: x.upper().replace("H", ":"))

In [24]:
for i in range(0, len(df)):
    mensagem = df['hora'][i]
    horas = re.findall(r'[0-9]{1,2}:[0-9]{1,2}', mensagem) #capturando horas HH:mm
    quantidade_horas = len(horas)
    
    if quantidade_horas == 1: #capturando apenas com uma horas escrita
        df['hora'][i] = horas[0]
    else: #restante com nenhuma ou mais horas (geralmente pq digitou a hora de entrada e saida {informação inutil})
        df['hora'][i] = ''
        df['controle'][i] = 'HORA'
        
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'HORA']])
print("probs:", len(df_prob))
sample_prob('HORA')

df = df[df['controle'] != 'HORA']
print("ok:", len(df)) 

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,54,96.428571
HORA,2,3.571429


probs: 42
ok: 637


In [25]:
#adicionando 0 no primeiro algorismo dos valores h:mm
for i in range(0, len(df)):
    if df['hora'][i][1] == ":":
        df['hora'][i] = "0"+df['hora'][i]
        
df['hora'].unique()

array(['15:37', '15:45', '15:40', '15:55', '11:00', '16:10', '16:15',
       '16:25', '16:35', '05:00', '16:40', '16:50', '16:20', '06:00',
       '17:08', '14:40', '17:00', '12:00', '18:00', '18:40', '19:10',
       '19:15', '19:20', '05:50', '19:45', '19:40', '19:50', '20:05',
       '20:00', '20:12', '20:18', '20:20', '20:40', '20:30', '20:50',
       '21:08', '21:35', '21:40', '21:45', '21:50', '22:10', '22:20',
       '00:10', '22:40', '16:00', '02:40', '03:20', '04:20', '04:10',
       '05:15', '04:40', '05:20', '05:10', '05:30', '06:30', '06:20',
       '08:00', '07:00', '09:00', '05:35', '05:40', '09:55', '11:06',
       '11:10', '10:15', '11:15', '09:30', '13:00', '14:10', '14:20',
       '14:15', '22:50', '14:45', '15:20', '15:30', '09:40', '15:43',
       '04:15', '15:35', '15:50', '05:45', '13:30', '16:30', '17:10',
       '19:00', '19:30', '20:10', '20:25', '20:45', '10:00', '20:57',
       '21:10', '21:20', '23:10', '23:40', '04:30', '04:50', '06:35',
       '08:35', '08:

# Hora final:

In [26]:
#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

In [27]:
df['hora'].unique()

array(['15:37', '15:45', '15:40', '15:55', '11:00', '16:10', '16:15',
       '16:25', '16:35', '05:00', '16:40', '16:50', '16:20', '06:00',
       '17:08', '14:40', '17:00', '12:00', '18:00', '18:40', '19:10',
       '19:15', '19:20', '05:50', '19:45', '19:40', '19:50', '20:05',
       '20:00', '20:12', '20:18', '20:20', '20:40', '20:30', '20:50',
       '21:08', '21:35', '21:40', '21:45', '21:50', '22:10', '22:20',
       '00:10', '22:40', '16:00', '02:40', '03:20', '04:20', '04:10',
       '05:15', '04:40', '05:20', '05:10', '05:30', '06:30', '06:20',
       '08:00', '07:00', '09:00', '05:35', '05:40', '09:55', '11:06',
       '11:10', '10:15', '11:15', '09:30', '13:00', '14:10', '14:20',
       '14:15', '22:50', '14:45', '15:20', '15:30', '09:40', '15:43',
       '04:15', '15:35', '15:50', '05:45', '13:30', '16:30', '17:10',
       '19:00', '19:30', '20:10', '20:25', '20:45', '10:00', '20:57',
       '21:10', '21:20', '23:10', '23:40', '04:30', '04:50', '06:35',
       '08:35', '08:

In [28]:
#criando uma coluna datetime para fazer calculos de horas e minutos
df['hora2'] = df['hora'].apply(lambda x: datetime.strptime(x, '%H:%M'))

#verificando se esta tudo certo
print(df['hora2'].dt.year.unique())
print(df['hora2'].dt.month.unique())
print(df['hora2'].dt.day.unique())
print(df['hora2'].dt.hour.unique())

for i in range(0, len(df)):
    minuto = int(df['hora'][i][3:])
    
    if minuto != 0 and minuto != 30:
        if minuto < 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (0) subtraindo quantos minutos tem entre o momento e HH:00
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-minuto))[11:-3]
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (30) acrecentando quantos minutos faltam para HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(30-minuto)))[11:-3]
            else:
                df['controle'][i] = 'Hora_final'
                
        elif minuto > 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (30) subtraindo quando minutos tem entre o momento e HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-(minuto-30)))[11:-3]   
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (60) acrecentando quantos minutos faltam para HH:60
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(60-minuto)))[11:-3]            
            else:
                df['controle'][i] = 'Hora_final'
        else:
            df['controle'][i] = 'Hora_final'
            
    elif minuto == 0 or minuto == 30:
        df['hora'][i] = str(df['hora2'][i])[11:-3] 
        
    else:
        df['controle'][i] = 'Hora_final'

df['hora'].unique()

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'Hora_final']])
print("probs:", len(df_prob))
sample_prob('Hora_final')

df = df[df['controle'] != 'Hora_final']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

[1900]
[1]
[1]
[15 11 16  5  6 17 14 12 18 19 20 21 22  0  2  3  4  8  7  9 10 13 23]
probs: 42
ok: 637


# Setor:

In [29]:
#reconhendo em qual periodo do dia aconteceu o plantão
df['ENTRADA'] = ''
df['SAIDA'] = ''

df['PERIODO'] = ''

for i in range(0, len(df)):
    minuto = int(df['hora'][i][:2])
    
    if df['tipo'][i] == 'ENTRADA':
        df['ENTRADA'][i] = df['hora'][i]
        if minuto < 7: #periodo 6h
            df['PERIODO'][i] = '6h'
            df['SAIDA'][i] = '07:00'
        elif minuto >= 7 and minuto < 13: #periodo manha
            df['PERIODO'][i] = 'manha'
            df['SAIDA'][i] = '13:00'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            df['PERIODO'][i] = 'tarde'
            df['SAIDA'][i] = '19:00'
        else:
            df['controle'][i] = 'HORARIOS'
    elif df['tipo'][i] == 'SAIDA':
        df['SAIDA'][i] = df['hora'][i]
        if minuto >= 7 and minuto < 13: #periodo manha
            df['ENTRADA'][i] = '07:00'
            df['PERIODO'][i] = 'manha'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            if df['hora'][i] == '13:00':
                df['SAIDA'][i] = '13:00'
                df['ENTRADA'][i] = '07:00'
                df['PERIODO'][i] = 'manha'
            else:
                df['ENTRADA'][i] = '13:00'
                df['PERIODO'][i] = 'tarde'
        elif minuto >= 19: #periodo noturno
            df['ENTRADA'][i] = '19:00'
            df['PERIODO'][i] = 'noturno saida'
        else:
            df['controle'][i] = 'HORARIOS'
    else:
        df['controle'][i] = 'HORARIOS'
        
count_freq(df, 'PERIODO')
count_freq(df, 'controle')

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'HORARIOS']])
print("probs:", len(df_prob))
sample_prob('HORARIOS')

df = df[df['controle'] != 'HORARIOS']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
PERIODO,,
6h,209,32.810047
tarde,167,26.216641
noturno saida,159,24.960754
manha,97,15.227630
,5,0.784929


,Freq,Perc
controle,,
DESLOCADO,54,91.525424
HORARIOS,5,8.474576


probs: 47
ok: 632


# DURAÇÃO DO PLANTÃO

In [30]:
df['DURAÇÃO'] = ''

for i in range(0, len(df)):
    diferenca_tempo = datetime.strptime(df['SAIDA'][i], '%H:%M') - datetime.strptime(df['ENTRADA'][i], '%H:%M')
    
    #diferenca_tempo vem em formato estranho, tem-se que transformar em segundos antes
    diferenca_tempo_s = diferenca_tempo.seconds
    horas = '0' + str(int((diferenca_tempo_s/60)/60)) #capturando horas
    minutos = str(int((diferenca_tempo_s/60)%60))  #capturando minutos
    
    if len(minutos) == 1:
        minutos = '0' + minutos
    
    df['DURAÇÃO'][i] = horas + ':' + minutos
    
df['DURAÇÃO'][df['DURAÇÃO'] == '00:00'] = '06:00'    

df['DURAÇÃO'].unique()

array(['03:00', '02:00', '03:30', '04:00', '01:00', '04:30', '05:00',
       '06:00', '00:30', '01:30', '02:30', '05:30'], dtype=object)

In [31]:
df[['mensagem','hora','ENTRADA','SAIDA','DURAÇÃO']][df['DURAÇÃO'] == '06:00']

,mensagem,hora,ENTRADA,SAIDA,DURAÇÃO
24,19/06 JABAQUARA ELAINE SAÍDA 18:40,19:00,19:00,19:00,06:00
92,20/06 VNS RAFAEL TADEU SAÍDA 7:00 ( EXTRA ) ...,07:00,07:00,07:00,06:00
112,20/06 - ANALIA MAURÍCIO ENTRADA 13:00 ( A P...,13:00,13:00,19:00,06:00
145,20/06 JABAQUARA - BRUCE ENTRADA 07:00,07:00,07:00,13:00,06:00
150,20/06 ANÁLIA MAURÍCIO SAÍDA 19:00,19:00,19:00,19:00,06:00
210,21/06 HSI - COELHO ENTRADA 7:00 ( ASSUMO RO...,07:00,07:00,13:00,06:00
213,20/06 ANALIA - BARBOSA ENTRADA 13:00,13:00,13:00,19:00,06:00
236,21/06 JABAQUARA CLARA SAÍDA 12:40,13:00,07:00,13:00,06:00
239,21/06 ITAIM - VINICIUS SAÍDA 13:00,13:00,07:00,13:00,06:00
253,21/06 VILLA HELENA ENTRADA 13:00,13:00,13:00,19:00,06:00


In [32]:
df['setor'] = ''

for i in range(0, len(df)):
    if df['sem_ou_fds'][i] == 'semana':
        if df['PERIODO'][i] == '6h':
            df['setor'][i] = 'SEMANA - 6HS'
        elif df['PERIODO'][i] == 'manha':
            df['setor'][i] = 'SEMANA - MANHA - C.CIRURGICO'
        elif df['PERIODO'][i] == 'tarde':
            df['setor'][i] = 'SEMANA - TARDE - C.CIRURGICO'
        elif df['PERIODO'][i] == 'noturno saida':
            df['setor'][i] = 'SEMANA - NOTURNO - SAIDA'
        else:
            df['setor'][i] = 'VERIFICAR'

    elif df['sem_ou_fds'][i] == 'fds': 
        if df['PERIODO'][i] == '6h':
            df['setor'][i] = 'FDS - 6HS'
        elif df['PERIODO'][i] == 'manha':
            df['setor'][i] = 'FDS - C.CIRURGICO ROTINA'
        elif df['PERIODO'][i] == 'tarde':
            df['setor'][i] = 'FDS - SAIDA'
        elif df['PERIODO'][i] == 'noturno saida':
            df['setor'][i] = 'FDS - PLANTAO'
        else:
            df['setor'][i] = 'VERIFICAR'  

    else:
        df['setor'][i] = 'VERIFICAR'
        
count_freq(df, 'setor')

,Freq,Perc
setor,,
SEMANA - 6HS,165,26.107595
SEMANA - NOTURNO - SAIDA,151,23.892405
SEMANA - TARDE - C.CIRURGICO,125,19.778481
SEMANA - MANHA - C.CIRURGICO,71,11.234177
FDS - 6HS,44,6.962025
FDS - SAIDA,42,6.645570
FDS - C.CIRURGICO ROTINA,26,4.113924
FDS - PLANTAO,8,1.265823


In [33]:
print("probs:", len(df_prob))
print("ok:", len(df))

porcentagem = round((len(df)/(len(df)+len(df_prob)))*100, 2)
print('Porcentagem de registros corrigidas pelo o algoritmo:', porcentagem)

probs: 47
ok: 632
Porcentagem de registros corrigidas pelo o algoritmo: 93.08


# Ultimos ajustes:

# Distancias

In [34]:
for i in range(0, len(df)):
    mensagem = df['mensagem'][i].upper()
    regex_D = re.findall('D[0-9]', mensagem)
    
    if len(regex_D) == 0:
        continue
    else:
        df['controle'][i] == 'DISTANCIAS'

In [35]:
for i in range(0, len(df)):
    if df['DURAÇÃO'][i] == "00:00":
        df['controle'][i] = "DISTANCIAS"
        
count_freq(df, 'controle')

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'DISTANCIAS']])
print("probs:", len(df_prob))
sample_prob('DISTANCIAS')

df = df[df['controle'] != 'DISTANCIAS']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,54,100.0


probs: 47
ok: 632


In [36]:
# Cria uma nova Série sem as listas
serie_sem_listas = pd.Series([x for x in df['unidade'] if not isinstance(x, list)])

# Imprime os valores únicos da nova Série
print(serie_sem_listas.unique())

['ITAIM' 'MORUMBI' 'ANALIA' 'AVICCENA' 'OSASCO' 'VILLA' 'SANTA' 'JABA'
 'ALPHA' 'VNS' 'MATERNIDADE']


In [37]:
# Cria uma lista com apenas os valores que são listas
listas = [x for x in df['unidade'] if isinstance(x, list)]

# Converte as listas em tuplas e adiciona elas a um conjunto para obter apenas as listas distintas
listas_distintas = set(tuple(lista) for lista in listas)

# Imprime as listas distintas
for lista in listas_distintas:
    print(list(lista))


['OSASCO', 'JABA']
['VNS', 'MORUMBI']
['OSASCO', 'ANALIA']
['SANTA', 'VNS']
['ALPHA', 'VNS']
['ALPHA', 'JABA']
['OSASCO', 'VILLA']
['ALPHA', 'ANALIA']
['MORUMBI', 'ITAIM']
['OSASCO', 'MORUMBI']
['ALPHA', 'VILLA']
['VILLA', 'ALPHA']
['ITAIM', 'OSASCO']
['MORUMBI', 'VNS']
['MORUMBI', 'ANALIA']
['VNS', 'ITAIM']
['ITAIM', 'VNS']
['OSASCO', 'JABA', 'OSASCO']
['OSASCO', 'ITAIM']
['OSASCO', 'MATERNIDADE']
['SANTA', 'ITAIM']
['VNS', 'ANALIA']
['VILLA', 'VILA']
['ANALIA', 'ALPHA']
['ANALIA', 'ITAIM']
['JABA', 'VILA']
['ANALIA', 'MATERNIDADE']
['OSASCO', 'VNS']
['ANALIA', 'OSASCO']


In [38]:
df['setor'] = df['setor'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))

#problemas com santa isabel 
#df['setor'][(df['unidade'] == "SANTA ISABEL") & (df['setor'] == "FDS - SAIDA")] = "FDS - NOTURNO - SAIDA"
df['setor'][(df['unidade'] == "SANTA ISABEL") & (df['setor'] == "FDS - C.CIRURGICO ROTINA")] = "FDS - MANHA - C.CIRURGICO"

In [39]:
df['nome'] = df['nome'].apply(lambda x: x.replace('  ',' '))

#df['UNIDADE'] = df['UNIDADE'].apply(lambda x: x.replace('JABAQUARAQUARA','JABAQUARA'))
#df['UNIDADE'].unique()

# Salvando:

In [40]:
df['periodo'] = np.nan
df['periodo'][df['PERIODO'] == '6h'] = 1
df['periodo'][df['PERIODO'] == 'manha'] = 2
df['periodo'][df['PERIODO'] == 'tarde'] = 3
df['periodo'][df['PERIODO'] == 'noturno saida'] = 4

In [41]:
df = df[['chat', 'dia', 'nome', 'mensagem', 'controle', 'data', 'unidade',
       'extra', 'tipo', 'ENTRADA', 'SAIDA', 'DURAÇÃO', 'PERIODO','periodo',
         'setor', 'sem_ou_fds']]

In [42]:
df.head()

,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,SAIDA,DURAÇÃO,PERIODO,periodo,setor,sem_ou_fds
0,[19/06/2024 15:37:24] Sergio Zamith: 19/06 Ita...,19/06/2024,SERGIO ZAMITH,19/06 ITAIM - ZAMITH SAÍDA 15:37,NaN,19/06/2024,ITAIM,NaN,SAIDA,13:00,16:00,03:00,tarde,3.0,SEMANA - TARDE - C.CIRURGICO,semana
1,[19/06/2024 15:47:24] Rodrigo Miyashiro Nunes ...,19/06/2024,RODRIGO MIYASHIRO NUNES DOS SANTOS,19/06 MORUMBI - MIYASHIRO SAIDA 15:45,NaN,19/06/2024,MORUMBI,NaN,SAIDA,13:00,16:00,03:00,tarde,3.0,SEMANA - TARDE - C.CIRURGICO,semana
2,[19/06/2024 15:52:56] Pedro Paulo Silva Farah:...,19/06/2024,PEDRO PAULO SILVA FARAH,19/06 VNS ( DESLOCADO MORUMBI ) PEDRO SAIDA...,DESLOCADO,19/06/2024,"[VNS, MORUMBI]",NaN,SAIDA,13:00,16:00,03:00,tarde,3.0,SEMANA - TARDE - C.CIRURGICO,semana
3,[19/06/2024 15:54:34] Patrick Alexander Sauer ...,19/06/2024,PATRICK ALEXANDER SAUER SCHUES,19/06 ANÁLIA - PATRICK SAÍDA 15:55,NaN,19/06/2024,ANALIA,NaN,SAIDA,13:00,16:00,03:00,tarde,3.0,SEMANA - TARDE - C.CIRURGICO,semana
4,[19/06/2024 16:14:37] Antonio Alvaro Dias Lima...,19/06/2024,ANTONIO ALVARO DIAS LIMA DA CUNHA NETO,19/06 CENTRAL - ÁLVARO ENTRADA 11:00,NaN,19/06/2024,AVICCENA,NaN,ENTRADA,11:00,13:00,02:00,manha,2.0,SEMANA - MANHA - C.CIRURGICO,semana


In [43]:
count_freq(df, 'controle')

,Freq,Perc
controle,,
DESLOCADO,54,100.0


# Ultimos ajustes nos deslocados:

In [44]:
df['controle'][(df['PERIODO'] == '6h') & (df['controle'] == 'DESLOCADO')] = 'DESLOCADO 6H'
count_freq(df, 'controle')

,Freq,Perc
controle,,
DESLOCADO,35,64.814815
DESLOCADO 6H,19,35.185185


In [45]:
df['unidade2'] = df['unidade']

for i in range(0, len(df)):
    if isinstance(df.loc[i, 'unidade'], list):
        df['unidade'][i] = df['unidade'][i][0]

df['unidade'][df['unidade'] == 'VILA'] = 'VILLA'
df['unidade'][df['unidade'] == 'JABA'] = 'JABAQUARA'
df['unidade'][df['unidade'] == 'SANTA'] = 'SANTA ISABEL'

sorted(df['unidade'].unique())
count_freq(df, 'unidade')

,Freq,Perc
unidade,,
ANALIA,97,15.348101
JABAQUARA,94,14.873418
OSASCO,89,14.082278
MORUMBI,88,13.924051
VILLA,78,12.341772
ITAIM,74,11.708861
SANTA ISABEL,39,6.170886
VNS,35,5.537975
AVICCENA,17,2.689873


# Salvando tudo:

In [46]:
salvando_nome_arquivo('df_gabs.xlsx', df)
print('.')
salvando_nome_arquivo('df_giov.xlsx', df_prob)

632
.
47


# DISPENSADOS:

In [47]:
df = pd.read_excel('df_giov.xlsx')[['chat','dia','nome','mensagem']]
df = df.drop_duplicates(subset=['chat'])
df = df.sort_values(['nome', 'dia'])
print(len(df))
df.sample(3)

47


,chat,dia,nome,mensagem
20,[23/06/2024 12:17:30] Amanda Kiyoko Souza Koss...,23/06/2024,AMANDA KIYOKO SOUZA KOSSAKA,23/06 ANÁLIA - AMANDA ENTRADA 08:20 ( D3 VI...
9,[21/06/2024 09:47:51] Gabriel de Aguiar Santan...,21/06/2024,GABRIEL DE AGUIAR SANTANA,20/06 ITAIM - GABRIEL SAÍDA 02:20 ( DISTÂNC...
24,[23/06/2024 18:03:11] Gustavo Marques da Silva...,23/06/2024,GUSTAVO MARQUES DA SILVA,23/06 OSASCO - MARQUES SAÍDA 13:30 ( D2 JAB...


In [48]:
#RETIRANDO REGISTROS DE DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE OU DISPENSA

df['controle'] = np.nan

#preparando para reconhecer corretamente

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DISPENSADO|SAUDE|DISPENSADA', mensagem)
    quantidade_observacoes = len(observacoes)
    
    if quantidade_observacoes == 0:
        df['controle'][i] = 'PROB'
        
df['controle'].value_counts() 

#salvando
df_prob = df[df['controle'] == 'PROB']
df_prob.reset_index(drop=True, inplace=True)
print("unidades probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("unidade ok:", len(df)) 

unidades probs: 37
unidade ok: 10


In [49]:
#DATA DO PLANTÃO
df['controle'] = ''
df['data'] = ''

for i in range(0, len(df)):
    lista_datas = re.findall(r'[0-9]{1,2}\/[0-9]{1,2}', df['mensagem'][i]) #regex dd/mm
    quantidade_itens = len(lista_datas)
    
    if quantidade_itens == 1: #capturar apenas registros com uma data apenas
        df['data'][i] = lista_datas[0]
        df['controle'][i] = 'NORM'
    else: #restante com nenhuma ou mais de uma data
        df['controle'][i] = 'PROB'

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] != 'NORM']])
df_prob.reset_index(drop=True, inplace=True)
print("data probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("datas ok:", len(df)) 

df['data'] = df['data']+"/2024"
sorted(df['data'].unique())

data probs: 37
datas ok: 10


['19/06/2024', '20/06/2024', '22/06/2024', '24/06/2024', '25/06/2024']

In [50]:
#retirando a data da mensagem para não atrapalhar no reconhecimento de hora
df['hora'] = ''

for i in range(0, len(df)):
    df['hora'][i] = df['mensagem'][i].replace(df['data'][i], '').strip() #substituindo a data por vazio
    
#substituindo "h" por ":" para capturar horarios 11h22
df['hora'] = df['hora'].apply(lambda x: x.upper().replace("H", ":"))

#Retirando aqueles que contem horas no registro da mensagem
for i in range(0, len(df)):
    mensagem = df['hora'][i]
    horas = re.findall(r'[0-9]{1,2}:[0-9]{1,2}', mensagem) #capturando horas HH:mm
    quantidade_horas = len(horas)
    
    if quantidade_horas >= 1: #capturando apenas com uma horas escrita
        df['hora'][i] = horas[0]
        df['controle'][i] = 'PROB'
    else: #restante com nenhuma ou mais horas (geralmente pq digitou a hora de entrada e saida {informação inutil})
        df['hora'][i] = ''
        df['controle'][i] = 'OK'
        
print(df['controle'].value_counts())

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] != 'OK']])
df_prob.reset_index(drop=True, inplace=True)
print("probs:", len(df_prob))

df = df[df['controle'] == 'OK']
df.reset_index(drop=True, inplace=True)
print("ok:", len(df)) 

controle
OK    10
Name: count, dtype: int64
probs: 37
ok: 10


In [51]:
print(len(df_prob))
print(len(df))

37
10


In [52]:
df = df.drop_duplicates(subset=['chat'])

os.remove("df_giov.xlsx")
salvando_nome_arquivo('df_giov.xlsx', df_prob)
salvando_nome_arquivo('df_dispensados.xlsx', df)

37
10


# ===========================================

In [53]:
print(len(pd.DataFrame({'chat': read_file('_chat.txt') })))
print(len(pd.read_excel('df_dispensados.xlsx')))
print(len(pd.read_excel('df_gabs.xlsx')))
print(len(pd.read_excel('df_giov.xlsx')))

679
10
632
37
